In [1]:
import sys
sys.path.insert(0 , '.')
import reactome, Pnet
import pandas as pd

%load_ext autoreload
%autoreload 2

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
net = reactome.ReactomeNetwork(genes_of_interest=['BRCA1' , 'BRCA2' , 'CCNH' , 'CCNT1' ] , n_levels=5)

In [3]:
gene_mask , pathway_masks = net.get_masks()

In [4]:
model = Pnet.PNET(reactome_network=net, input_dim=3 , output_dim=10)

In [5]:
model

PNET(
  (activation): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=4, bias=True)
    (1): MaskedLinear(in_features=4, out_features=12, bias=True)
    (2): MaskedLinear(in_features=12, out_features=25, bias=True)
    (3): MaskedLinear(in_features=25, out_features=24, bias=True)
    (4): MaskedLinear(in_features=24, out_features=13, bias=True)
  )
  (skip): ModuleList(
    (0): Linear(in_features=4, out_features=10, bias=True)
    (1): Linear(in_features=12, out_features=10, bias=True)
    (2): Linear(in_features=25, out_features=10, bias=True)
    (3): Linear(in_features=24, out_features=10, bias=True)
    (4): Linear(in_features=13, out_features=10, bias=True)
  )
)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

# Step 1: Generate random data
# Let's create a dataset with 100 samples, each having 5 features
input_size = 3
output_size = 16
num_samples = 100

device = torch.device('cuda')

# Random input data and target values
X = torch.randn(num_samples, input_size).to(device)
y = torch.randn(num_samples, output_size).to(device)

model = Pnet.PNET(reactome_network=net, input_dim=3 , output_dim=16).to(device)

# Step 3: Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

# Step 4: Train the model
num_epochs = 100

for epoch in range(num_epochs):
    # Forward pass: compute the model output
    # Check if the tensor elements are close to zero
    outputs = model(X)
    loss = criterion(outputs, y)
    
    # Backward pass: compute gradients
    optimizer.zero_grad()
    loss.backward()
    
    # Update the weights
    optimizer.step()

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    
# Example output after training
print("\nTraining completed.")
example_input = torch.randn(1, input_size).to(device)
example_output = model(example_input)
print("Example input:\n", example_input)
print("Example output:\n", example_output)


Epoch [10/100], Loss: 1.0476
Epoch [20/100], Loss: 1.0474
Epoch [30/100], Loss: 1.0473
Epoch [40/100], Loss: 1.0471
Epoch [50/100], Loss: 1.0469
Epoch [60/100], Loss: 1.0467
Epoch [70/100], Loss: 1.0466
Epoch [80/100], Loss: 1.0464
Epoch [90/100], Loss: 1.0462
Epoch [100/100], Loss: 1.0461

Training completed.
Example input:
 tensor([[-0.5910, -0.5932,  0.9589]], device='cuda:0')
Example output:
 tensor([[ 0.0062,  0.1031,  0.1174,  0.0339, -0.0412, -0.0397,  0.1149, -0.0813,
          0.0408, -0.0029, -0.0264,  0.1323,  0.0407, -0.0191,  0.0879, -0.0251]],
       device='cuda:0', grad_fn=<DivBackward0>)


In [62]:
import captum

In [65]:
dl = captum.attr.DeepLift(model)
gene_importances = dl.attribute((X), target=0)

/Users/barryryan/anaconda3/lib/python3.11/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/Users/barryryan/anaconda3/lib/python3.11/site-packages/captum/attr/_core/deep_lift.py:304: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(


In [67]:
gene_importances.shape

torch.Size([100, 3])

In [58]:
model.deepLIFT(X , 0)

AttributeError: 'Tensor' object has no attribute 'x'

In [71]:

model.interpret_flag=True
ig = captum.attr.IntegratedGradients(model)

ig_attr, delta = ig.attribute((X), return_convergence_delta=True, target=0)
gene_importances = ig_attr
gene_importances = pd.DataFrame(gene_importances.detach().numpy())




In [76]:
model.layers[1]

MaskedLinear(in_features=4, out_features=12, bias=True)

In [77]:

model.interpret_flag=True
layer_importance_scores = []
cond = captum.attr.LayerConductance(model, model.layers[1])  # ReLU output of masked layer at each level
cond_vals = cond.attribute((X), target=0)
cols = [self.reactome_network.pathway_encoding.set_index('ID').loc[col]['pathway'] for col in self.reactome_network.pathway_layers[0].index]
cond_vals_genomic = pd.DataFrame(cond_vals.detach().numpy(),
                                 columns=cols,
                                 index=test_dataset.input_df.index)
pathway_imp_by_target = cond_vals_genomic
layer_importance_scores.append(pathway_imp_by_target)

for i, level in enumerate(self.layers):
    cond = captum.attr.LayerConductance(self, level.pathway_layer)  # ReLU output of masked layer at each level
    cond_vals = cond.attribute((test_dataset.x, test_dataset.additional), target=target_class)
    cols = [self.reactome_network.pathway_encoding.set_index('ID').loc[col]['pathway'] for col in self.reactome_network.pathway_layers[i].columns]
    cond_vals_genomic = pd.DataFrame(cond_vals.detach().numpy(),
                                     columns=cols,
                                     index=test_dataset.input_df.index)
    pathway_imp_by_target = cond_vals_genomic
    layer_importance_scores.append(pathway_imp_by_target)
self.interpret_flag=False

NameError: name 'self' is not defined

In [80]:
cond_vals.shape

torch.Size([100, 12])

In [ ]:


def layerwise_activation(self, test_dataset, target_class=0):
    self.interpret_flag=True
    layer_importance_scores = []
    for i, level in enumerate(self.layers):
        act = captum.attr.LayerActivation(self, level.pathway_layer)
        act_vals = act.attribute((test_dataset.x, test_dataset.additional), attribute_to_layer_input=True)
        cols = [self.reactome_network.pathway_encoding.set_index('ID').loc[col]['pathway'] for col in self.reactome_network.pathway_layers[i].index]
        act_vals_genomic = pd.DataFrame(act_vals.detach().numpy(),
                                        columns=cols,
                                        index=test_dataset.input_df.index)
        pathway_imp_by_target = act_vals_genomic
        layer_importance_scores.append(pathway_imp_by_target)
    self.interpret_flag=False
    return layer_importance_scores